In [ ]:
#Tensorflow
!pip3 install tensorflow==2.4.0

#Other dependencies
!pip install keras==2.4.3 numpy==1.19.3 pillow==7.0.0 scipy==1.4.1 h5py==2.10.0 matplotlib==3.3.2 opencv-python keras-resnet==0.2.0

#ImageAI
!pip install imageai --upgrade

     |████████████████████████████████| 394.7 MB 18 kB/s 
     |████████████████████████████████| 14.8 MB 60.1 MB/s 
     |████████████████████████████████| 462 kB 65.8 MB/s 
     |████████████████████████████████| 3.8 MB 55.1 MB/s 
     |████████████████████████████████| 2.9 MB 59.4 MB/s 
     |████████████████████████████████| 132 kB 64.9 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68714 sha256=803d121169d656bd6711be1d2579f569f3dbba439fd480d9730e40acf2e8230d
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting

     |████████████████████████████████| 14.9 MB 15.1 MB/s 
     |████████████████████████████████| 2.1 MB 58.1 MB/s 
     |████████████████████████████████| 11.6 MB 57.0 MB/s 
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20486 sha256=dbfe60f4c9bd39a686dd36a6d45d7c239a573c548f84d85c4d41b8251a416a6b
  Stored in directory: /root/.cache/pip/wheels/bd/ef/06/5d65f696360436c3a423020c4b7fd8c558c09ef264a0e6c575
Successfully built keras-resnet
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: matplotlib
    Found existing installation: 

     |████████████████████████████████| 160 kB 7.4 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install detecto

In [ ]:
#import libraries

import numpy as np
import cv2
import matplotlib.pyplot as plt
from math import sqrt
import sys
import cv2 as cv
import matplotlib

from detecto import core, utils, visualize
from detecto.visualize import show_labeled_image, plot_prediction_grid
from torchvision import transforms

import warnings
warnings.filterwarnings('ignore')

In [ ]:
cd /content/drive/MyDrive/OxfordTown/Social-Distance-Surveillance-master/train-validation-torch

/content/drive/MyDrive/OxfordTown/Social-Distance-Surveillance-master/train-validation-torch


In [ ]:
#function to add padding to the image
def padding(image,n=100):
    
    #inversion of colours as cv2 takes BGR format by default
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    #add padding(border) to images
    constant= cv.copyMakeBorder(image,1*n,1*n,1*n,1*n,cv.BORDER_CONSTANT) 
    
    return constant


#function to rotate image to a particular
def rotate(img, angle, rotPoint=None):
    (height,width) = img.shape[:2]
    
    if rotPoint is None:
        rotPoint = (width//2, height//2) #centre as the rotation point
       
    rotMat = cv.getRotationMatrix2D(rotPoint, angle, 0.85)
    dimensions = (width,height)
    
    return cv.warpAffine(img, rotMat, dimensions)


#trained resnet model
model = core.Model.load('model_weights_25.pth', ['Person']) #non augmented model


#funtion to compute the coordinates in the bird's eye view
def compute_point_perspective_transformation(matrix,boxes):
    list_downoids = [[box[0]+(box[2]-box[0])/2+300, box[3]+300] for box in boxes]
    #list_downoids = [[[box[0]+300, box[1]+300],[box[0]+box[2]+300,box[1]+300],[box[0]+300,box[1]+box[3]+300],[box[0]+box[2]+300,box[1]+box[3]+300]] for box in boxes]
    #list_downoids = [[box[4]+300, box[5]+box[3]//2+300] for box in boxes]
    list_points_to_detect = np.float32(list_downoids).reshape(-1, 1, 2)
    transformed_points = cv2.perspectiveTransform(list_points_to_detect, matrix)
    transformed_points_list = list()
    for i in range(0,transformed_points.shape[0]):
        transformed_points_list.append([transformed_points[i][0][0],transformed_points[i][0][1]])
    return np.array(transformed_points_list).astype('int')


#function to find euclidean distance between two points
def eucledian_distance(point1, point2):
    x1,y1 = point1
    x2,y2 = point2
    return sqrt((x1-x2)**2 + (y1-y2)**2)


#function to return a list of red and green boxes
def get_red_green_boxes(distance_allowed,birds_eye_points,boxes):
    red_boxes = []
    green_boxes = []

    new_boxes = [tuple(box) + tuple(result) for box, result in zip(boxes, birds_eye_points)]
    for i in range(0, len(new_boxes)-1):
            for j in range(i+1, len(new_boxes)):
                cxi,cyi = new_boxes[i][4:]
                cxj,cyj = new_boxes[j][4:]
                distance = eucledian_distance([cxi,cyi], [cxj,cyj])
                if distance < distance_allowed:
                    red_boxes.append(new_boxes[i])
                    red_boxes.append(new_boxes[j])

    green_boxes = list(set(new_boxes) - set(red_boxes))
    red_boxes = list(set(red_boxes))

    return (green_boxes, red_boxes)


#plot perspective view of violators (red) and non - violators (green)
def get_perspective_view_image(green_box, red_box,eye_view_height,eye_view_width):
    blank_image = cv2.imread('black_background.png')   
    for point in green_box:
        cv2.circle(blank_image,tuple([int(point[0]+(point[2]-point[0])/2),int(point[3])]),20,(0,255,0),-1)
    for point in red_box:
        cv2.circle(blank_image,tuple([int(point[0]+(point[2]-point[0])/2),int(point[3])]),20,(0,0,255),-1)
    blank_image = cv2.resize(blank_image,(eye_view_width,eye_view_height))
    return blank_image


#draw red rectangles around violators and green rectangle around non - violators
def get_red_green_box_image(new_box_image,green_box,red_box):
    for point in green_box:
        cv2.rectangle(new_box_image,(point[0],point[1]),(point[2],point[3]),(0, 255, 0), 2)
    for point in red_box:
        cv2.rectangle(new_box_image,(point[0],point[1]),(point[2],point[3]),(0, 0, 255), 2)
    return new_box_image

In [ ]:
#reading the video file
video = cv2.VideoCapture('TownCentre.avi')
writer = None
frame_number = 0

#printing the heading
print('%-20s%-26s%-26s%-26s' % ('Processing Frame','|Total Detected Person','|Red Markerd Person','|Green Marked Person'))

while True:
  
  #ret is a boolean variable that returns true if the frame is available and frame stores the image
  ret,frame = video.read()

  if not ret:
    break

  image_height, image_width = frame.shape[:2]    #image dimensions
  #image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  image = frame.copy()                           #copying the image

  #add fixed test for demarcation
  cv2.putText(image,'Store',(185,70),cv2.FONT_HERSHEY_SIMPLEX,1.5,(255,0,0),4,2)
  cv2.putText(image,'Entrance',(735,190),cv2.FONT_HERSHEY_SIMPLEX,1.5,(255,0,0),4,2)
  cv2.putText(image,'Road',(510,871),cv2.FONT_HERSHEY_SIMPLEX,1.5,(255,0,0),4,2)
  cv2.putText(image,'Bench',(569,437),cv2.FONT_HERSHEY_SIMPLEX,1.5,(255,0,0),4,2)
  cv2.putText(image,'Sitting Place',(1501,459),cv2.FONT_HERSHEY_SIMPLEX,1.5,(255,0,0),4,2)
  cv2.putText(image,'Black Pillar',(1192,829),cv2.FONT_HERSHEY_SIMPLEX,1.5,(255,0,0),4,2)
  cv2.putText(image,'ATM',(1659,70),cv2.FONT_HERSHEY_SIMPLEX,1.5,(255,0,0),4,2)  

  img = padding(image,300)                       #padding

  # Coordinates that you want to Perspective Transform
  pts1 = np.float32([[1660,2],[2917,1184],[2,33],[482,2070]])
  # Size of the Transformed Image
  pts2 = np.float32([[0,0],[1920,0],[0,1080],[1920,1080]])
  #Warp Images
  H_matrix = cv2.getPerspectiveTransform(pts1,pts2)      #getting transformed matrix
  warped = cv2.warpPerspective(img,H_matrix,(1920,1080)) #getting warped image

  fin = rotate(warped,282, (warped.shape[:2][1]//1.6, warped.shape[:2][0]//2)) #rotating the warped image

  #predicting people in the image using model - resnet
  predictions = model.predict(image)
  labels, boxes, scores = predictions

  thresh=0.6 #set threshold for scores
  
  filtered_indices=np.where(scores>thresh)
  filtered_scores=scores[filtered_indices]
  filtered_boxes=boxes[filtered_indices]
  num_list = filtered_indices[0].tolist()
  filtered_labels = [labels[i] for i in num_list]
  boxes = filtered_boxes.numpy() #convert tensor to numpy

  birds_eye_points = compute_point_perspective_transformation(H_matrix, boxes) #getting the coordinates of people in warped image

  #reading each coordinates of people and drawing a circle at the bottom centre of people
  for i in range(0,len(birds_eye_points)):
    cv2.circle(warped,tuple([birds_eye_points[i][0],birds_eye_points[i][1]]), 15, (0, 0, 255), -1)

  min_distance = 122.12339 #it is the threshold pixel distance which people should maintain to avoid social distance violation

  green_box, red_box = get_red_green_boxes(min_distance, birds_eye_points, boxes) #getting the co-ordinates of the green box(non-violators) and red box(violators)

  perspective_view_image = get_perspective_view_image(green_box, red_box, eye_view_height=image_height, eye_view_width=image_width) #getting perspective image with green and red dots

  box_red_green_image = get_red_green_box_image(image,green_box,red_box) #getting image with green and red boxes

  #resizing the image to 1920 x 1080
  perspective_view_image = cv2.resize(perspective_view_image, (800, 1080)) 
  box_red_green_image = cv2.resize(box_red_green_image, (1120, 1080)) 

  #combining images
  combined_image_final = np.concatenate((perspective_view_image,box_red_green_image), axis=1)
  
  #counting the frame number
  frame_number += 1

  #prints the frame number, number of detected persons, no. of detected ppl violating, no. of detected ppl who are not violating social distance
  sys.stdout.write('%-20i|%-25i|%-25i|%-25i\n' % (frame_number,len(boxes),len(red_box),len(green_box)))

  if frame_number >=1525: #the required number of output frames is mentioned here and the while loop breaks when the output video has reached the given number of frames
    break

  if writer is None: #this if-loop runs for first time only
    fourcc = cv2.VideoWriter_fourcc(*"DIVX") #FourCC is a 4-byte code used to specify the video codec
    writer = cv2.VideoWriter('OUTPUT_withoutAudio.avi', fourcc, 25, (combined_image_final.shape[1], combined_image_final.shape[0]), True) #file name, video codex, frame rate, shape, color or gray scale

  #demarcation

  #
  writer.write(combined_image_final) #adding image results in the output video

  del image,fin,warped,combined_image_final,perspective_view_image,box_red_green_image #deleting the stored images so that new image can be loaded again

print(' ')
writer.release()
video.release()

Processing Frame    |Total Detected Person    |Red Markerd Person       |Green Marked Person      
1                   |12                       |4                        |8                        
2                   |12                       |4                        |8                        
3                   |12                       |4                        |8                        
4                   |13                       |6                        |7                        
5                   |13                       |6                        |7                        
6                   |13                       |6                        |7                        
7                   |14                       |7                        |7                        
8                   |13                       |6                        |7                        
9                   |15                       |8                        |7                        
10        